
# 順方向伝播、逆方向伝播、および計算グラフ

:label: `sec_backprop`

これまでのところ、ミニバッチ確率的勾配降下法を使用してモデルをトレーニングしてきました。ただし、アルゴリズムを実装するときは、モデルを介した*順伝播*に関連する計算のみを懸念していました。勾配を計算する段階になったら、深層学習フレームワークによって提供される逆伝播関数を呼び出すだけです。

勾配の自動計算 (自動微分) により、深層学習アルゴリズムの実装が大幅に簡素化されます。自動微分が行われる前は、複雑なモデルに小さな変更を加える場合でも、複雑な導関数を手動で再計算する必要がありました。驚くべきことに、学術論文では更新ルールを導き出すために多くのページを割り当てる必要があることがよくあります。興味深い部分に集中できるように自動微分に依存し続ける必要がありますが、深層学習についての浅い理解を超えたければ、これらの勾配が内部でどのように計算されるかを知っておく必要があります。

このセクションでは、*逆方向伝播*(より一般的には*バックプロパゲーション*と呼ばれます) の詳細を詳しく説明します。テクニックとその実装の両方について洞察を伝えるために、いくつかの基本的な数学と計算グラフに依存します。まず、重み減衰を伴う 1 隠れ層 MLP ($\ell_2$ 正則化、後続の章で説明) に焦点を当てて説明します。

## 順伝播

*順伝播*(または*順方向パス*) は、ニューラル ネットワークの中間変数 (出力を含む) を入力層から出力層に順番に計算して保存することを指します。ここでは、1 つの隠れ層を持つニューラル ネットワークの仕組みを段階的に見ていきます。退屈に思えるかもしれないが、ファンクの巨匠ジェームス・ブラウンの永遠の言葉を借りれば、「ボスになるためには代償を払わなければならない」ということだ。

簡単にするために、入力例が $\mathbf{x}\in \mathbb{R}^d$ であり、隠れ層にバイアス項が含まれていないと仮定します。ここで中間変数は次のとおりです。

 $$\mathbf{z}= \mathbf{W}^{(1)} \mathbf{x},$$

ここで $\mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$ は隠れ層の重みパラメータです。中間変数 $\mathbf{z}\in \mathbb{R}^h$ を活性化関数 $\phi$ を通して実行した後、長さ $h$ の隠れた活性化ベクトルを取得します。

 $$\mathbf{h}= \phi (\mathbf{z}).$$

隠れ層の出力 $\mathbf{h}$ も中間変数です。出力層のパラメーターの重みが $\mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$ のみであると仮定すると、次の式で出力層変数を取得できます。長さ $q$ のベクトル:

 $$\mathbf{o}= \mathbf{W}^{(2)} \mathbf{h}.$$

損失関数が $l$、ラベル例が $y$ であると仮定すると、単一のデータ例の損失項を計算できます。

 $$L = l(\mathbf{o}, y).$$

後で紹介する $\ell_2$ 正則化の定義によれば、ハイパーパラメータ $\lambda$ が与えられた場合、正則化項は次のようになります。

 $$s = \frac{\lambda}{2} \left(|\mathbf{W}^{(1)}|_F^2 + |\mathbf{W}^{(2)}|_F^2\右)、$$ :eqlabel: `eq_forward-s`

ここで、行列のフロベニウス ノルムは、行列をベクトルに平坦化した後に適用される $\ell_2$ ノルムです。最後に、特定のデータ例におけるモデルの正規化損失は次のようになります。

 $$J = L + s.$$

以下の説明では、$J$ を*目的関数*と呼びます。

## 順伝播の計算グラフ

*計算グラフ*をプロットすると、計算内の演算子と変数の依存関係を視覚化するのに役立ちます。 :numref: `fig_forward`は、上で説明した単純なネットワークに関連付けられたグラフが含まれています。四角は変数を示し、円は演算子を示します。左下隅は入力を表し、右上隅は出力を表します。 (データ フローを示す) 矢印の方向は主に右向きと上向きであることに注意してください。

![](http://d2l.ai/_images/forward.svg) :label: `fig_forward`

## 誤差逆伝播法

*バックプロパゲーションとは、*ニューラル ネットワーク パラメーターの勾配を計算する方法を指します。つまり、この方法は、微積分の*連鎖規則*に従って、出力層から入力層まで逆の順序でネットワークを横断します。このアルゴリズムは、いくつかのパラメーターに関する勾配を計算する際に必要な中間変数 (偏導関数) を保存します。関数 $\mathsf{Y}=f(\mathsf{X})$ と $\mathsf{Z}=g(\mathsf{Y})$ があり、入力と出力 $\mathsf{ X}、\mathsf{Y}、\mathsf{Z}$ は任意の形状のテンソルです。連鎖則を使用すると、次のように $\mathsf{X}$ に関する $\mathsf{Z}$ の導関数を計算できます。

 $$\frac{\partial \mathsf{Z}}{\partial \mathsf{X}} = \text{prod}\left(\frac{\partial \mathsf{Z}}{\partial \mathsf{Y} }, \frac{\partial \mathsf{Y}}{\partial \mathsf{X}}\right).$$

ここでは、転置や入力位置の交換などの必要な操作が実行された後、$\text{prod}$ 演算子を使用して引数を乗算します。ベクトルの場合、これは簡単です。単に行列と行列の乗算です。高次元テンソルの場合は、適切な対応物を使用します。演算子 $\text{prod}$ は、すべての表記のオーバーヘッドを非表示にします。

計算グラフが :numref: `fig_forward`にある、1 つの隠れ層を持つ単純なネットワークのパラメーターが $\mathbf{W}^{(1)}$ と $\mathbf{W}^{(2)} であることを思い出してください。 $。バックプロパゲーションの目的は、勾配 $\partial J/\partial \mathbf{W}^{(1)}$ と $\partial J/\partial \mathbf{W}^{(2)}$ を計算することです。これを達成するために、連鎖規則を適用し、各中間変数とパラメーターの勾配を順番に計算します。計算グラフの結果から始めてパラメータに向かって進む必要があるため、計算の順序は順伝播で実行される順序とは逆になります。最初のステップは、損失項 $L$ と正則化項 $s$ に関する目的関数 $J=L+s$ の勾配を計算することです。

 $$\frac{\部分 J}{\部分 L} = 1 ; \text{そして} ; \frac{\partial J}{\partial s} = 1.$$

次に、出力層 $\mathbf{o}$ の変数に対する目的関数の勾配を連鎖規則に従って計算します。

 $$ \frac{\partial J}{\partial \mathbf{o}} = \text{prod}\left(\frac{\partial J}{\partial L}, \frac{\partial L}{\partial \mathbf{o}}\right) = \frac{\partial L}{\partial \mathbf{o}} \in \mathbb{R}^q。 $$

次に、両方のパラメーターに関して正則化項の勾配を計算します。

 $$\frac{\partial s}{\partial \mathbf{W}^{(1)}} = \lambda \mathbf{W}^{(1)} ; \text{そして} ; \frac{\partial s}{\partial \mathbf{W}^{(2)}} = \lambda \mathbf{W}^{(2)}.$$

これで、出力層に最も近いモデル パラメーターの勾配 $\partial J/\partial \mathbf{W}^{(2)} \in \mathbb{R}^{q \times h}$ を計算できるようになりました。 。連鎖ルールを使用すると、次の結果が得られます。

 $$\frac{\partial J}{\partial \mathbf{W}^{(2)}}= \text{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \mathbf{o}}{\partial \mathbf{W}^{(2)}}\right) + \text{prod}\left(\frac{\partial J}{\partial s} , \frac{\partial s}{\partial \mathbf{W}^{(2)}}\right)= \frac{\partial J}{\partial \mathbf{o}} \mathbf{h}^\ top + \lambda \mathbf{W}^{(2)}.$$ :eqlabel: `eq_backprop-Jh`

 $\mathbf{W}^{(1)}$ に関する勾配を取得するには、出力層に沿って隠れ層まで逆伝播を続ける必要があります。隠れ層出力 $\partial J/\partial \mathbf{h} \in \mathbb{R}^h$ に関する勾配は次の式で与えられます。

 $$ \frac{\partial J}{\partial \mathbf{h}} = \text{prod}\left(\frac{\partial J}{\partial \mathbf{o}}, \frac{\partial \ mathbf{o}}{\partial \mathbf{h}}\right) = {\mathbf{W}^{(2)}}^\top \frac{\partial J}{\partial \mathbf{o}} 。 $$

活性化関数 $\phi$ は要素ごとに適用されるため、中間変数 $\mathbf{z}$ の勾配 $\partial J/\partial \mathbf{z} \in \mathbb{R}^h$ を計算するには、次のことが必要です。 $\odot$ で表す要素ごとの乗算演算子を使用します。

 $$ \frac{\partial J}{\partial \mathbf{z}} = \text{prod}\left(\frac{\partial J}{\partial \mathbf{h}}, \frac{\partial \ mathbf{h}}{\partial \mathbf{z}}\right) = \frac{\partial J}{\partial \mathbf{h}} \odot \phi&#39;\left(\mathbf{z}\right) 。 $$

最後に、入力層に最も近いモデル パラメーターの勾配 $\partial J/\partial \mathbf{W}^{(1)} \in \mathbb{R}^{h \times d}$ を取得できます。連鎖則によれば、次のようになります。

 $$ \frac{\partial J}{\partial \mathbf{W}^{(1)}} = \text{prod}\left(\frac{\partial J}{\partial \mathbf{z}}, \frac{\partial \mathbf{z}}{\partial \mathbf{W}^{(1)}}\right) + \text{prod}\left(\frac{\partial J}{\partial s} , \frac{\partial s}{\partial \mathbf{W}^{(1)}}\right) = \frac{\partial J}{\partial \mathbf{z}} \mathbf{x}^\トップ + \lambda \mathbf{W}^{(1)}。 $$

## ニューラルネットワークのトレーニング

ニューラル ネットワークをトレーニングする場合、順方向伝播と逆方向伝播は相互に依存します。特に、順伝播の場合、計算グラフを依存関係の方向にたどり、そのパス上のすべての変数を計算します。これらは、グラフ上の計算順序が逆になるバックプロパゲーションに使用されます。

前述の単純なネットワークを例として説明します。一方では、順伝播中の正則化項 :eqref: `eq_forward-s`の計算は、モデル パラメーター $\mathbf{W}^{(1)}$ および $\mathbf{W}^{(2) の現在の値に依存します。 )}$。これらは、最新の反復におけるバックプロパゲーションに応じた最適化アルゴリズムによって与えられます。一方、逆伝播中のパラメータ :eqref: `eq_backprop-Jh`の勾配計算は、順伝播によって与えられる隠れ層出力 $\mathbf{h}$ の現在値に依存します。

したがって、ニューラル ネットワークをトレーニングするときは、モデル パラメーターが初期化された後、順伝播と逆伝播を交互に実行し、逆伝播によって与えられる勾配を使用してモデル パラメーターを更新します。逆伝播では、計算の重複を避けるために、順伝播で保存された中間値が再利用されることに注意してください。結果の 1 つは、バックプロパゲーションが完了するまで中間値を保持する必要があることです。これは、トレーニングに単純な予測よりもはるかに多くのメモリが必要になる理由の 1 つでもあります。さらに、このような中間値のサイズは、ネットワーク層の数とバッチ サイズにほぼ比例します。したがって、より大きなバッチ サイズを使用してより深いネットワークをトレーニングする*と、メモリ不足*エラーが発生しやすくなります。

## まとめ

順伝播では、ニューラル ネットワークによって定義された計算グラフ内の中間変数が順次計算され、保存されます。入力層から出力層に進みます。バックプロパゲーションでは、ニューラル ネットワーク内の中間変数とパラメーターの勾配を逆の順序で順次計算して保存します。深層学習モデルをトレーニングする場合、順伝播と逆伝播は相互に依存しており、トレーニングには予測よりもはるかに多くのメモリが必要です。

## 演習
1. あるスカラー関数 $f$ への入力 $\mathbf{X}$ が $n \times m$ 行列であると仮定します。 $\mathbf{X}$ に対する $f$ の勾配の次元は何ですか?
1. このセクションで説明するモデルの隠れ層にバイアスを追加します (正則化項にバイアスを含める必要はありません)。<ol><li>対応する計算グラフを描画します。
1. 順方向伝播方程式と逆方向伝播方程式を導出します。
1. 複数の GPU に分割できますか?
1. より小さいミニバッチでトレーニングする場合の利点と欠点は何ですか?

[ディスカッション](https://discuss.d2l.ai/t/102)
